In [1]:
import pygrib
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import random
from collections import defaultdict
import datetime

In [2]:
input_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"
grib_files = sorted(glob.glob(os.path.join(input_dir, "ST4.*.01h")))

output_dir = "/N/project/Zli_lab/gongg/stage4_data/stage4_nc"

print(datetime.datetime.now())
for grb_file in grib_files[60000:]:
    base_name = os.path.basename(grb_file)  
  
    time_str = base_name.split(".")[1]   
    
    time_dt = datetime.datetime.strptime(time_str, "%Y%m%d%H")


    grbs = pygrib.open(grb_file)
    
    selected_grb = grbs.select(name="Total Precipitation")[0]
    
    data, lat, lon = selected_grb.data()
    grbs.close()

    
    array_2d = data.filled(np.nan)

    
    ds = xr.Dataset(
        {
            "tp": (["y", "x"], array_2d)
        },
        coords={
            "lat": (["y", "x"], lat),
            "lon": (["y", "x"], lon),
           
            "time": time_dt
        }
    )

    
    nc_filename = os.path.join(output_dir, f"{time_str}.nc")
    ds.to_netcdf(nc_filename)

print(datetime.datetime.now())

2025-03-20 22:15:27.971840
2025-03-20 22:56:56.473764


In [4]:
input_dir = "/N/project/Zli_lab/Data/Observations/stage4_jja_processed"
output_dir = "/N/project/Zli_lab/gongg/stage4_data/stage4_nc"


grib_files = sorted(glob.glob(os.path.join(input_dir, "ST4.*.01h")))
input_times = {os.path.basename(f).split(".")[1]: f for f in grib_files}


nc_files = sorted(glob.glob(os.path.join(output_dir, "*.nc")))
output_times = {os.path.splitext(os.path.basename(f))[0] for f in nc_files}


missing_times = set(input_times.keys()) - output_times

In [5]:
sorted_missing = sorted(missing_times)


for time_str in sorted_missing[15000:]:
    grb_file = input_times[time_str]
    try:
        time_dt = datetime.datetime.strptime(time_str, "%Y%m%d%H")
        grbs = pygrib.open(grb_file)
        selected_grb = grbs[1]  # 注意：pygrib的索引从1开始
        data, lat, lon = selected_grb.data()
        grbs.close()

        array_2d = data.filled(np.nan)

        ds = xr.Dataset(
            {
                "tp": (["y", "x"], array_2d)
            },
            coords={
                "lat": (["y", "x"], lat),
                "lon": (["y", "x"], lon),
                "time": time_dt
            }
        )

        nc_filename = os.path.join(output_dir, f"{time_str}.nc")
        ds.to_netcdf(nc_filename)
    except Exception:
        pass